In [2]:
#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

import time #библиотека для установки паузы между запросами к сайту

#общие библиотеки
import pandas as pd
import numpy as np

import re

In [3]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Chrome/108.0.0.0'}
urllib3.disable_warnings()

In [4]:
url_star = 'https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaMIBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALlmsecBsACAdICJDFmMDhmM2QxLTMxYTAtNDMzMC05ODYwLTllNWU2M2EyMDZiN9gCBeACAQ&aid=304142&ss=Paris%2C+Ile+de+France%2C+France&ssne=Amsterdam&ssne_untouched=Amsterdam&efdco=1&lang=en-us&src=searchresults&dest_id=-1456928&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=d7cc527a560c0272&ac_meta=GhBkN2NjNTI3YTU2MGMwMjcyIAAoATICZW46BVBhcmlzQABKAFAA&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=ht_id%3D204%3Bclass%3D'
url_all_star = []

for i in range(2):
    url_all_star.append(url_star + str(i+4) + '&order=distance_from_search')

In [5]:
urls_hotel_4 = []
urls_hotel_5 = []

for star in range(2):
    time.sleep(3)  
    if star == 0:
        
        for i in range(0, 526, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_4.append(a['href'])
        time.sleep(3) 
    if star == 1:
        for i in range(0, 101, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_5.append(a['href'])
        time.sleep(3)
    


print(len(urls_hotel_4))
print(len(urls_hotel_5))

527
105


In [6]:
urls_hotel_4_en = []
urls_hotel_5_en = []

    
for i in range(len(urls_hotel_4)):
    parts = urls_hotel_4[i].split('hapos=' + str(i+1))
    urls_hotel_4_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])
    
for i in range(len(urls_hotel_5)):
    parts = urls_hotel_5[i].split('hapos=' + str(i+1))
    urls_hotel_5_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])


print(len(urls_hotel_4_en))
print(len(urls_hotel_5_en))

527
105


In [7]:
data_array_full = np.empty(4, 'str')

tags = ['span', 'div', 'div']
classes = ['hp_address_subtitle js-hp_address_subtitle jq_tooltip', 'd8eab2cf7f c90c0a70d3 db63693c62',
           'b5cd09854e f0d4d6a2f5 e46e88563a']

all_star_url = [urls_hotel_4_en, urls_hotel_5_en]


count_star_0 = 3

for urls in all_star_url:
    count_star_0 += 1
    for url in urls:
        try:
            response = req.get(url, headers=header, timeout=50, verify=False)
            page = BeautifulSoup(response.text,'html.parser')
        except:
            print('ошибка запроса')
        
    
        data = []
        data.append(count_star_0)
        for i in range(3):
            try:
                elem = page.find_all(tags[i], classes[i])
                elem = elem[0].text
            except:
                elem = np.NaN
        
                
            data.append(elem)
            
        data_array = np.asarray(data).reshape(1,4)
        data_array_full = np.row_stack((data_array_full,data_array))
    
        time.sleep(3)

ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса


In [50]:
columns = np.array(['hotel_star', 'hotel_address','reviews_count','quality'])
hotel_paris = pd.DataFrame(data=data_array_full, columns=columns)
hotel_paris.replace('nan', np.NaN, inplace=True)
hotel_paris = hotel_paris.dropna(thresh=2).reset_index(drop=True)
hotel_paris

,hotel_star,hotel_address,reviews_count,quality
0,,,,
1,4,"\n11 Rue Du Temple, 4th arr., 75004 Paris, Fra...",554 reviews,Very Good
2,4,"\n19 Rue du Bourg Tibourg, 4th arr., 75004 Par...",207 reviews,Excellent
3,4,"\nRue Sainte-Croix de la Bretonnerie, 4th arr....",17 reviews,Very Good
4,4,"\n58 Rue du Roi de Sicile, 4th arr., 75004 Par...",368 reviews,Excellent
...,...,...,...,...
628,5,"\n10 Place De Mexico, 16th arr., 75016 Paris, ...",56 reviews,Good
629,5,"\n3 Rue De Boulainvilliers, 16th arr., 75016 P...",559 reviews,Very Good
630,5,"\n5 place du Chancelier Adenauer, 16th arr., 7...",89 reviews,Wonderful
631,5,"\n1-7 Rue Jean Richepin, 16th arr., 75116 Pari...",432 reviews,Wonderful


In [51]:
hotel_df = pd.read_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/hotels.csv', sep=',')
hotel_df.shape

(386803, 57)

In [52]:
hotel_paris['hotel_address'] = hotel_paris['hotel_address'].apply(
    lambda x: re.sub("[^0-9a-zA-Z]"," ",x).strip().lower().replace(' ', ''))

In [57]:
hotel_paris['hotel_address']

0                                                       
1                    11ruedutemple4tharr75004parisfrance
2              19ruedubourgtibourg4tharr75004parisfrance
3      ruesaintecroixdelabretonnerie4tharr75004parisf...
4               58rueduroidesicile4tharr75004parisfrance
                             ...                        
628               10placedemexico16tharr75016parisfrance
629         3ruedeboulainvilliers16tharr75016parisfrance
630    5placeduchancelieradenauer16tharr75016parisfrance
631             17ruejeanrichepin16tharr75116parisfrance
632          13ruenungesseretcoli16tharr75016parisfrance
Name: hotel_address, Length: 605, dtype: object

In [59]:
hotel_df['hotel_address'] = hotel_df['hotel_address'].apply(
    lambda x: x.lower().replace(' ', ''))

In [60]:
hotel_df['hotel_address']

0         strattonstreetmayfairwestminsterboroughlondonw...
1         130134southamptonrowcamdenlondonwc1b5afunitedk...
2                   151bisruederennes6tharr75006parisfrance
3                216avenuejeanjaures19tharr75019parisfrance
4                      molenwerf11014agamsterdamnetherlands
                                ...                        
386798    9knaresboroughplacekensingtonandchelsealondons...
386799    landstraerhauptstrae15503landstrae1030viennaau...
386800      2931gowerstreetcamdenlondonwc1e6hgunitedkingdom
386801    31greatcumberlandplacewestminsterboroughlondon...
386802    25courtfieldgardenskensingtonandchelsealondons...
Name: hotel_address, Length: 386803, dtype: object

In [61]:
hotel_df = hotel_df.merge(
                hotel_paris,
                on='hotel_address',
                how='left')

In [62]:
null = hotel_df[hotel_df['city_hotel'] == 'Paris'].isnull().mean() * 100
null_per = null[null > 0]
null_per

hotel_star       14.015168
reviews_count    14.510373
quality          14.510373
dtype: float64

In [54]:
dupl_columns = list(hotel_paris.columns) 

mask = hotel_paris.duplicated(subset=dupl_columns) 
hotel_duplicates = hotel_paris[mask]
print(f'Число найденных дубликатов: {hotel_duplicates.shape[0]}')

Число найденных дубликатов: 26


In [55]:
hotel_paris = hotel_paris.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {hotel_paris.shape[0]}')

Результирующее число записей: 607


In [56]:
hotel_paris = hotel_paris.drop_duplicates('hotel_address')

In [65]:
hotel_paris.to_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/parsing_hotels/hotel_paris.csv')